In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Dropout, Flatten
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [ ]:
##Using GUP
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 设置 GPU 设备可见
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU 设备配置成功.")
    except RuntimeError as e:
        print(e)

In [ ]:
#Load data
# import geometry input data and  output data
xdata=pd.read_csv("C:/Users/kunkun/Desktop/PythenData/train/geometry.csv",header=None)
xdata=xdata.values
ydata=pd.read_csv("C:/Users/kunkun/Desktop/PythenData/train/target.csv",header=None)
ydata=ydata.values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.1)

In [ ]:
class LearningRateLogger(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        
        current_step = self.model.optimizer.iterations.numpy()
        current_learning_rate = lr_schedule(current_step)
        print(f"lr {epoch}: {current_learning_rate}")


initial_learning_rate = 0.005
decay_steps = 2000
decay_rate = 0.95
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps, decay_rate, staircase=False)

#  Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

learning_rate_logger = LearningRateLogger()


###R2
class R2Logger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.r2_values_train = []
        self.r2_values_val = []

    def on_epoch_end(self, epoch, logs=None):
        
        y_train_pred = self.model.predict(X_train)
        y_val_pred = self.model.predict(X_val)

        
        r2_train = r2_score(y_train, y_train_pred)
        r2_val = r2_score(y_val, y_val_pred)

        
        self.r2_values_train.append(r2_train)
        self.r2_values_val.append(r2_val)

        
        print(f"R² of training {epoch + 1}: {r2_train}")
        print(f"R² of validation {epoch + 1}: {r2_val}")
        

r2_logger = R2Logger()

##MAPE and RMSE
class MetricsLogger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.mape_values_train = []
        self.mape_values_val = []
        self.rmse_values_train = []
        self.rmse_values_val = []

    def on_epoch_end(self, epoch, logs=None):
        # 计算训练集和验证集上的预测值
        y_train_pred = self.model.predict(X_train)
        y_val_pred = self.model.predict(X_val)

        # 计算MAPE和RMSE
        mape_train = mean_absolute_percentage_error(y_train, y_train_pred)
        mape_val = mean_absolute_percentage_error(y_val, y_val_pred)
        rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
        rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))

        # 存储MAPE和RMSE值
        self.mape_values_train.append(mape_train)
        self.mape_values_val.append(mape_val)
        self.rmse_values_train.append(rmse_train)
        self.rmse_values_val.append(rmse_val)

        # 输出MAPE和RMSE
        print(f"MAPE of training {epoch + 1}: {mape_train}")
        print(f"MAPE of validation {epoch + 1}: {mape_val}")
        print(f"RMSE of training {epoch + 1}: {rmse_train}")
        print(f"RMSE of validation {epoch + 1}: {rmse_val}")
        
MAPE_RMSE = MetricsLogger()

In [ ]:
#neutral network
with tf.device('GPU:0'):
    model = Sequential()

    model.add(Dense(100, input_dim=xdata.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(140, activation='linear'))
    model.summary()
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae'], loss_weights=[0.8, 0.2])

In [ ]:
#Trianing
with tf.device('GPU:0'):
    history = model.fit(x=X_train, y=y_train, validation_split=0.1, epochs=3000, batch_size=64,  verbose=1, callbacks=[learning_rate_logger, r2_logger, MAPE_RMSE])